# Setup

In [ ]:
using ProgressMeter, Distributed
# Add workers for parallel processing
prs = addprocs(;topology=:master_worker, exeflags="-O3")

@everywhere using Pkg; @everywhere Pkg.activate(".")

# Load relevant code on all processes
using SteadyStateStability
@everywhere using SteadyStateStability

In [ ]:
# Define data location for all processes
if Sys.islinux()
    diskLoc = "/media/allen/Seagate Backup Plus Drive/"
elseif Sys.iswindows()
   diskLoc = "F:\\"
end

rootdir = joinpath(diskLoc, "projects", "Arm-role-stability")

# Read all trials
sstrials = readsstrials(rootdir);

In [ ]:
[ (Subject=r[1], numtrials=length(r[2])) for r in [ (sub, findall(t -> t.subject == sub, sstrials)) for sub in 1:15 ] ]

## Main analysis

In [ ]:
# Setup progressmeter and lock
pdesc = "Processing data... "
p = Progress(length(sstrials)+1; desc=pdesc, barglyphs=BarGlyphs("[=>.]"))
uplock = ReentrantLock()

In [ ]:
# Update the progressmeter in a thread-safe manner
@everywhere function updateprogress()
    lock(uplock)
    next!(p)
    unlock(uplock)
    nothing
end

# This is used by the workers
@everywhere function analyzeandupdate(trial)
    numstrides = 125
    atrial = analyzetrial(trial, numstrides)
    
    # Tell the master process to update the progressmeter
    remotecall_wait(updateprogress,1)
    return atrial
end

In [ ]:
# Analyze all trials, don't bail on error
next!(p)
analyzedtrials = pmap(analyzeandupdate, sstrials; on_error=identity)
finish!(p)

In [ ]:
using Biomechanics

In [ ]:
# Check to see if any perturbations failed (to go back and see why they failed)
badtrials = findall(x -> !isa(x, AnalyzedSegment), analyzedtrials)
if !isempty(badtrials)
    @show badtrials
    @show analyzedtrials[badtrials]
    variables = [
        :lambdaS,
        :left_steplavg,
        :left_steplstd,
        :right_steplavg,
        :right_steplstd,
        :stepwavg,
        :stepwstd,
        :rsho_avgrom,
        :lsho_avgrom,
        :rsho_stdrom,
        :lsho_stdrom,
        :swingasym,
        :stepasym_spatial,
        :stepasym_temporal,
        :msdcrp_lsho_rhip,
        :msdcrp_rsho_lhip
    ]
    # NAN the bad trials
    analyzedtrials[badtrials] = [ AnalyzedSegment(Segment(trial, Dict{Symbol,Vector}(), SteadyStateSeg()), 
                                                          Dict{Symbol,Any}(( (var, NaN) for var in variables ))) 
                                  for trial in sstrials[badtrials] ]
end

In [ ]:
# We don't need the other workers anymore
rmprocs(prs)

## Results printing setup

In [ ]:
using Statistics, Dates, DelimitedFiles

In [ ]:
# Setup loop variables
variables = [
    :lambdaS,
    :left_steplavg,
    :left_steplstd,
    :right_steplavg,
    :right_steplstd,
    :stepwavg,
    :stepwstd,
    :rsho_avgrom,
    :lsho_avgrom,
    :rsho_stdrom,
    :lsho_stdrom,
    :swingasym,
    :stepasym_spatial,
    :stepasym_temporal,
    :msdcrp_lsho_rhip,
    :msdcrp_rsho_lhip
]
armconds = [ :none, :norm, :excess ]
shortarms = Dict(
    :norm => "_nr",
    :none => "_no",
    :excess => "_ex"
)
symconds = [ :sym, :asym ]
shortsym = Dict(
    :sym => "_sy,",
    :asym => "_as,"
)

subs = 1:15
numsubs = length(subs)
header = 4

# Initialize the results string
results = Vector{String}(undef, 1)
results[1] = "Analysis of stability, coordination, and symmetry during steady-state walking\nGenerated: $(now())\n"

In [ ]:
# Loop through all variables and conditions
# Beginning with the variables will keep all the variables chunked together, this will make
# the analysis in SPSS easier (all the relevant columns will be somewhat adjacent, limiting
# the need for lateral scrolling )
io = IOBuffer()
for vari in eachindex(variables)
    subresults = fill(",", 4)
    R = collect(1:15)
    _r = Array{Float64}(undef, numsubs)
    for arms in eachindex(armconds), symmetry in eachindex(symconds)
        # Only print the variable/condition if it is the first of its type
        subresults[1] *= prod([ arms, symmetry ] .== ones(Int,2)) ? string(variables[vari])*"," : ","
        subresults[2] *= (symmetry == one(Int)) ? string(armconds[arms], ",") : ","
        subresults[3] *= string(symconds[symmetry], ",")
        subresults[4] *= string(variables[vari])*shortarms[armconds[arms]]*shortsym[symconds[symmetry]]

        # Grab all trials for this combination of conditions
        relevant = findall(analyzedtrials) do ass
            ass.s.trial.conds[:arms] == armconds[arms] &&
            ass.s.trial.conds[:sym] == symconds[symmetry]
        end

        for sub in subs
            # Grab the appropriate trial for this subject
            t = filter(ass -> ass.s.trial.subject == sub, analyzedtrials[relevant])[1].results[variables[vari]]

            _r[sub] = t
        end
        
        R = [ R _r ]
    end

    writedlm(io, R, ',')

    results = [results; "\n\n"; subresults; String(take!(io))]
end

## Write to file

In [ ]:
(path, io) = mktemp()

for line in results
    println(io, line)
end

close(io)

resfn = joinpath(rootdir, "results", "steady-state.csv")

mv(path, resfn; force=true)